In [17]:
!pip install Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.5 MB/s eta 0:00:00


In [22]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import re
import numpy as np
import pandas as pd
import gensim
import nltk

from nltk.stem.snowball import SnowballStemmer

from gensim.corpora import Dictionary, MmCorpus
from gensim.models import ldamodel

from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.decomposition import PCA

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
import pandas as pd

In [24]:
news = pd.read_csv("news.csv", on_bad_lines='skip', header= None)

In [25]:
def preprocess_indo(sentence: list):
    # case Folding
    sentences = [sentence.lower() for sentence in sentence]
    # print(sentences)

    # tokenizer
    tokenizer = RegexpTokenizer(
        r'(?<!\S)[A-Za-z]+(?!\S)|(?<!\S)[A-Za-z]+(?=:(?!\S))')
    # hasil_token   = tokenizer.tokenize(sentences[1])
    token_sentence = [tokenizer.tokenize(sentence) for sentence in sentences]
    # print(token_sentence)

    words = """all,halaman,off"""
    word = words.split(",")

    # stopword
    wordlist = set(stopwords.words('indonesian') + word)
    sw_sentences = [[word for word in sentence if word not in wordlist]
                    for sentence in token_sentence]
    cleaned_sentences = [' '.join(sentence) for sentence in sw_sentences]
    # print(cleaned_sentences)

    # stemming
    stemmer = StemmerFactory().create_stemmer()
    stem_sentence = [stemmer.stem(sentence) for sentence in cleaned_sentences]
    # print(stem_sentence)
    return stem_sentence



def prerocessing_indo(sentences):
    # case Folding
    sentences = [sentence.lower() for sentence in sentences]

    # tokenizer
    tokenizer     = RegexpTokenizer(r'(?<!\S)[A-Za-z]+(?!\S)|(?<!\S)[A-Za-z]+(?=:(?!\S))')
    # hasil_token   = tokenizer.tokenize(sentences[1])
    token_sentence = [tokenizer.tokenize(sentence) for sentence in sentences]

    # stopword
    wordlist= set(stopwords.words('indonesian')+["all", "halaman", "off"])
    sw_sentences = [[word for word in sentence if word not in wordlist]for sentence in token_sentence]
    cleaned_sentences = [sentence for sentence in sw_sentences]

    sentence = cleaned_sentences
    return sentence

In [27]:
list_news = news.iloc[:,-1].values.tolist()

In [6]:
# list_news

In [28]:
clean_news = preprocess_indo(list_news)

In [31]:
pd.DataFrame(clean_news).to_csv("preprocess_news.csv")

In [29]:
# clean_news.to_csv("preprocess_news.csv")

AttributeError: ignored

In [35]:
clean_news[0].split(" ")

['cerita', 'bangun', 'konstruksi', 'waduk', 'cirata', 'ilmu', 'teknologi']

In [36]:
docs = [new.split(" ") for new in clean_news]

In [37]:
dictionary = Dictionary(docs)

# buang term yang:
# 1. muncul di kurang dari 10 dokumen
# 2. muncul di lebih dari 0.9*(total_dok) dokumen
dictionary.filter_extremes(no_below=30, no_above=100)

# ubah dictionary menjadi object bag-of-words reference
# ingat bahwa dalama LDA, dokumen diasumsikan dengan bag-of-words model
corpus = [dictionary.doc2bow(new) for new in docs]

In [39]:
# corpus

In [40]:
hdp_model = gensim.models.hdpmodel.HdpModel(corpus=corpus, id2word=dictionary)

In [41]:
# tampilkan topic matrix
topics_matrix = hdp_model.show_topics(num_topics = 5, num_words = 5,formatted=False)

for topic_no, topic_words in topics_matrix:

    print ('topic number: {}'.format(topic_no))

    # default
    for word, prob in topic_words:
        print (word, prob)
    print()

topic number: 0
indonesia 0.0027723669875771035
jokowi 0.0018806618959436288
bisnis 0.0018792353979208043
video 0.0018446501295692917
foto 0.0017695580373259928

topic number: 1
indonesia 0.003230799792673705
video 0.0025349324845739796
okezone 0.0019600069836557014
foto 0.0018687206324670112
bisnis 0.0018363187412348935

topic number: 2
indonesia 0.004550138377315827
bahas 0.0023337110379051235
mayat 0.0022829837228330254
bawa 0.001933495392641236
ruu 0.0019322199512847122

topic number: 3
indonesia 0.002372348021740747
mbappe 0.0020495578487102015
harga 0.0019397308008489598
foto 0.0017509813376993548
video 0.0016866522295139758

topic number: 4
indonesia 0.002814166031404612
hasil 0.0020451868328117983
vs 0.0018469970952697664
video 0.0017272766740258937
kemenag 0.0015545353780270109



In [18]:
list_news[0]

'Cerita Pembangunan Konstruksi Waduk Cirata - Ilmu dan Teknologi - '

In [13]:
clean_news[0]

['cerita', 'pembangunan', 'konstruksi', 'waduk', 'cirata', 'ilmu', 'teknologi']

In [15]:
hdp_model[dictionary.doc2bow(clean_news[0])]

[(51, 0.8344434153832727)]

In [17]:
hdp_model.print_topic(51)

'0.004*"indonesia" + 0.002*"pos" + 0.002*"studi" + 0.002*"destinasi" + 0.002*"foto" + 0.002*"video" + 0.002*"timnas" + 0.001*"laku" + 0.001*"sea" + 0.001*"live"'